#  ----------------- Doubts ------------------------

- We will need to transform all the categorical data into onehot encode? 
- Can't we use the id variable and note the name feature?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')
orders_products = pd.read_csv('order_products__train.csv')
sample = pd.read_csv('sample_submission.csv')

## Analysing the data

### Aisles

In [3]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
aisles.shape, len(aisles['aisle_id'].unique())

((134, 2), 134)

In [5]:
aisles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


## Departments

In [6]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


### Products

In [7]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [8]:
products.shape, len(products['product_id'].unique())

((49688, 4), 49688)

In [9]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


### Orders

In [10]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [11]:
orders.shape, len(orders['order_id'].unique())

((3421083, 7), 3421083)

### Orders_Products

In [12]:
orders_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [13]:
orders_products.shape, len(orders_products['order_id'].unique())

((1384617, 4), 131209)

In [14]:
orders_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           1384617 non-null  int64
 1   product_id         1384617 non-null  int64
 2   add_to_cart_order  1384617 non-null  int64
 3   reordered          1384617 non-null  int64
dtypes: int64(4)
memory usage: 42.3 MB


### Sample Submission

In [15]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   order_id  75000 non-null  int64 
 1   products  75000 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [16]:
sample.head()

,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


In [17]:
sample['products'][0]

'39276 29259'

### Creating a transaction Data Frame

The Idea of a transaction Data Frame is to have all the products for each order in just one row.

In [18]:
# Creating the df with all the informations that we need together
CT = orders_products.merge(products, on='product_id', how='left')

In [19]:
CT.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16
2,1,10246,3,0,Organic Celery Hearts,83,4
3,1,49683,4,0,Cucumber Kirby,83,4
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15


In [20]:
# Creating the quantity column
Prod_qtd = CT.groupby('product_name')['order_id'].count().reset_index()
Prod_qtd.rename(columns={'order_id':'quantity'}, inplace=True)
CT = CT.merge(Prod_qtd, on='product_name', how='left')

In [21]:
'''
grouping by order so we only have one row for each order and then creating a dictionary
in which the unique order_ids are the keys and all the products for each unique order are
the values (it is a list with all the products ordered in that especific order).
'''
product_id_dict= CT.groupby('order_id')['product_id'].apply(list).to_dict()
product_name_dict= CT.groupby('order_id')['product_name'].apply(list).to_dict()

In [22]:
# Creating the transaction df only with the coolumns
transaction = pd.DataFrame(columns=[['order_id', 'product_id', 'product_name']])

In [23]:
# transforming the list of product names into a single string:
for column, value in product_name_dict.items():
    product_name_dict[column] = ' '.join(str(e) for e in value)
    
# transforming the list of product ids into a single string:
for column, value in product_id_dict.items():
    product_id_dict[column] = ' '.join(str(e) for e in value)

In [24]:
# Filling our transaction DF with the values that we created in the dictionaries:
transaction['order_id'], transaction['product_id'], transaction['product_name'] = list(product_id_dict.keys()), list(product_id_dict.values()), list(product_name_dict.values())

In [25]:
transaction.head()

,order_id,product_id,product_name
0,1,49302 11109 10246 49683 43633 13176 47209 22035,Bulgarian Yogurt Organic 4% Milk Fat Whole Mil...
1,36,39612 19660 49235 43086 46620 34497 48679 46979,Grated Pecorino Romano Cheese Spring Water Org...
2,38,11913 18159 4461 21616 23622 32433 28842 42625...,Shelled Pistachios Organic Biologique Limes Or...
3,96,20574 30391 40706 25610 27966 24489 39275,Roasted Turkey Organic Cucumber Organic Grape ...
4,98,8859 19731 43654 13176 4357 37664 34065 35951 ...,Natural Spring Water Organic Orange Juice With...


## Market Basket Analysis

In [26]:
# Creating the dataset for the M.B.A.
pn = CT[CT['quantity']>=10000].groupby('order_id')['product_name'].apply(list).to_dict()
dataset = list(pn.values())

In [27]:
dataset[:3]

[['Bag of Organic Bananas'],
 ['Bag of Organic Bananas'],
 ['Bag of Organic Bananas', 'Organic Strawberries']]

In [28]:
# List with all the unique products:
unique_products = list(products['product_name'])


The first step is to count items or item combinations that have been frequently purchased together.

The relative frequency of these itemsets is called support. The word itemset can refer to both single and multiple items.

Transactional data is usually filtered for frequent itemsets with a certain minimum support level before association rules are mined.

This approach is called a priori because it calculates the probability that an item occurs in a frequent itemset when other items are present.

One of the easiest ways to calculate these support probabilities in Python is to use a package.

In this tutorial we will use the popular package mlxtend.

This package requires our transactional data in a one-hot-encoded format, where each column represents an item and each row represents a transaction.

Using the TransactionEncoder method from mlxtend, we can convert our list of list dataset into a one-hot-encoded dataset.

Run the following code to import the TransactionEncoder method and convert the dataset:

In [29]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
dataset_one_hot = te.fit(dataset).transform(dataset)
dataset_one_hot

array([[ True, False, False],
       [ True, False, False],
       [ True, False,  True],
       ...,
       [False, False,  True],
       [False,  True, False],
       [False, False,  True]])

You can see that this array contains only boolean True and False values to indicate whether an item was seen in a transaction or not (the actual order quantity does not matter in this approach).

To make the data easier to understand, let us bring back the column names and convert them into a Pandas DataFrame.

In [30]:
df_one_hot = pd.DataFrame(dataset_one_hot, columns=te.columns_)
df_one_hot

,Bag of Organic Bananas,Banana,Organic Strawberries
0,True,False,False
1,True,False,False
2,True,False,True
3,False,True,False
4,False,True,False
...,...,...,...
39827,True,False,False
39828,True,False,False
39829,False,False,True
39830,False,True,False


We can now proceed with the calculation of the frequent itemsets.

Let's assume we want to have a minimum support of 0.2 (that is, the item or item combination must appear in at least 20% of all transactions).

We can use the apriori method of mlxtend and set the parameter min_support = 0.2.

Run the following code to import the method and calculate the frequent itemsets:

In [31]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(df_one_hot, min_support=0.2, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False)

,support,itemsets
1,0.470125,(Banana)
0,0.388632,(Bag of Organic Bananas)
2,0.273499,(Organic Strawberries)


Association rule mining is not recommended for finding associations over very rare events. Therefore, computing the frequent itemset should be your first task. The level of support depends on your data and your business. You can experiment with different values to come up with good results.

### Association Rules

With the frequent_itemsets, we can now look for rules about which itemset (antecedent A) led to the purchase of another itemset (consequent C), also expressed as a rule A → C.

One measure for calculating these rules is confidence.

Confidence of a rule A → C is the probability of seeing C in a transaction given that it also contains A. If the confidence is 1, the consequent and antecedent always occur together.

If our minimum confidence level were 70%, there would be only one rule that could be derived from the frequent itemset above.

In [32]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [33]:
'''counted_data = CT[CT['reordered']==1].groupby('product_name')['order_id'].count().reset_index()

# plot the total number of each product ordered using a bar chart
plt.bar(counted_data['product_name'], counted_data['order_id'])
plt.xlabel('Product')
plt.ylabel('Number Ordered')
plt.title('Total Number of Each Product Ordered')
plt.show()'''

"counted_data = CT[CT['reordered']==1].groupby('product_name')['order_id'].count().reset_index()\n\n# plot the total number of each product ordered using a bar chart\nplt.bar(counted_data['product_name'], counted_data['order_id'])\nplt.xlabel('Product')\nplt.ylabel('Number Ordered')\nplt.title('Total Number of Each Product Ordered')\nplt.show()"

In [34]:
qtd = 100
CT[CT['quantity'] >= qtd]['product_name'].unique()

array(['Organic 4% Milk Fat Whole Milk Cottage Cheese',
       'Organic Celery Hearts', 'Cucumber Kirby', ...,
       'Sliced Beets No Salt Added', 'Fudge Mint Cookies',
       'Extra Sharp Cheddar Cheese'], dtype=object)

In [35]:
len(CT[CT['quantity'] >= qtd]['product_name'].unique())

2457

In [36]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [8, 20]

In [37]:
'''plt.bar(CT[CT['quantity'] >= qtd]['product_name'], CT[CT['quantity'] >= qtd]['quantity'])
plt.xlabel('Product')
plt.ylabel('Number Ordered')
plt.title('Total Number of Each Product Ordered')
plt.show()'''

"plt.bar(CT[CT['quantity'] >= qtd]['product_name'], CT[CT['quantity'] >= qtd]['quantity'])\nplt.xlabel('Product')\nplt.ylabel('Number Ordered')\nplt.title('Total Number of Each Product Ordered')\nplt.show()"

In [38]:
corr_matrix = CT.corr()

C:\Users\joaov\AppData\Local\Temp/ipykernel_35556/1487890889.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = CT.corr()
